# Regime Analysis

Analyze PPO performance across different market regimes: Bull/Bear and Volatility regimes.


In [ ]:
# Setup
import sys
from pathlib import Path
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set up paths
CURRENT_DIR = Path().resolve()
PROJECT_ROOT = CURRENT_DIR.parent
RESULTS_DIR = PROJECT_ROOT / "results"
EXPORTS_DIR = PROJECT_ROOT / "notebooks" / "exports"
EXPORTS_DIR.mkdir(exist_ok=True)

if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print(f"PROJECT_ROOT = {PROJECT_ROOT}")
print(f"RESULTS_DIR = {RESULTS_DIR}")

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)


## Load Regime Data


In [ ]:
# Load regime comparison CSV
regime_dir = RESULTS_DIR / "regimes"
regime_comparison_path = regime_dir / "regime_comparison.csv"
regime_metrics_path = regime_dir / "regime_metrics.json"

if regime_comparison_path.exists():
    regime_df = pd.read_csv(regime_comparison_path)
    print("Regime Comparison Data:")
    print(regime_df.to_string(index=False))
else:
    print("Regime comparison CSV not found")
    regime_df = None

# Also load JSON for additional details
if regime_metrics_path.exists():
    with open(regime_metrics_path, 'r') as f:
        regime_metrics_json = json.load(f)
    print("\nRegime Metrics (JSON):")
    print(json.dumps(regime_metrics_json, indent=2))
else:
    print("Regime metrics JSON not found")
    regime_metrics_json = None


## Visualize Regime Performance


In [ ]:
if regime_df is not None:
    # Create bar charts for each metric by regime
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    metrics_to_plot = ['total_return', 'annualized_return', 'sharpe', 'max_drawdown']
    
    for idx, metric in enumerate(metrics_to_plot):
        ax = axes[idx // 2, idx % 2]
        
        # Separate by regime type
        bull_bear_data = regime_df[regime_df['regime_type'] == 'bull_bear']
        vol_data = regime_df[regime_df['regime_type'] == 'volatility']
        
        x = np.arange(max(len(bull_bear_data), len(vol_data)))
        width = 0.35
        
        if not bull_bear_data.empty:
            ax.bar(x[:len(bull_bear_data)] - width/2, bull_bear_data[metric], 
                   width, label='Bull/Bear', alpha=0.8)
        if not vol_data.empty:
            ax.bar(x[:len(vol_data)] + width/2, vol_data[metric], 
                   width, label='Volatility', alpha=0.8)
        
        ax.set_ylabel(metric.replace("_", " ").title())
        ax.set_title(f'{metric.replace("_", " ").title()} by Regime')
        ax.set_xticks(x[:max(len(bull_bear_data), len(vol_data))])
        if not bull_bear_data.empty:
            labels = list(bull_bear_data['regime_label']) + [''] * (len(x) - len(bull_bear_data))
            ax.set_xticklabels(labels, rotation=45, ha='right')
        ax.legend()
        ax.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.savefig(EXPORTS_DIR / "regime_performance.png", dpi=150, bbox_inches='tight')
    plt.show()
    print(f"Saved regime performance plot to {EXPORTS_DIR / 'regime_performance.png'}")


In [ ]:
if regime_df is not None:
    # Create heatmap of metrics by regime
    pivot_df = regime_df.pivot_table(
        index='regime_label', 
        columns='regime_type', 
        values=['total_return', 'annualized_return', 'sharpe', 'max_drawdown'],
        aggfunc='first'
    )
    
    # Flatten column names
    pivot_df.columns = [f'{col[1]}_{col[0]}' for col in pivot_df.columns]
    
    fig, ax = plt.subplots(figsize=(10, 6))
    sns.heatmap(pivot_df.T, annot=True, fmt='.3f', cmap='RdYlGn', center=0, ax=ax)
    ax.set_title('Regime Performance Heatmap')
    plt.tight_layout()
    plt.savefig(EXPORTS_DIR / "regime_heatmap.png", dpi=150, bbox_inches='tight')
    plt.show()
    print(f"Saved regime heatmap to {EXPORTS_DIR / 'regime_heatmap.png'}")
    
    print("\nRegime Performance Table:")
    print(pivot_df.to_string())


## Export Regime Analysis Tables


In [ ]:
# Export utilities
def export_table_csv(df: pd.DataFrame, path: Path) -> None:
    """Export DataFrame to CSV."""
    df.to_csv(path, index=False)
    print(f"Exported table to {path}")

def export_table_latex(df: pd.DataFrame, path: Path, **kwargs) -> None:
    """Export DataFrame to LaTeX format."""
    latex_str = df.to_latex(index=False, **kwargs)
    with open(path, 'w') as f:
        f.write(latex_str)
    print(f"Exported LaTeX table to {path}")

def export_table_html(df: pd.DataFrame, path: Path, **kwargs) -> None:
    """Export DataFrame to HTML format."""
    html_str = df.to_html(index=False, **kwargs)
    with open(path, 'w') as f:
        f.write(html_str)
    print(f"Exported HTML table to {path}")

# Export regime comparison tables
if regime_df is not None:
    export_table_csv(regime_df, EXPORTS_DIR / "regime_comparison_analysis.csv")
    export_table_latex(regime_df, EXPORTS_DIR / "regime_comparison_analysis.tex", float_format="%.4f")
    export_table_html(regime_df, EXPORTS_DIR / "regime_comparison_analysis.html", classes='table table-striped')
    
    # Export separate tables for each regime type
    bull_bear_df = regime_df[regime_df['regime_type'] == 'bull_bear']
    vol_df = regime_df[regime_df['regime_type'] == 'volatility']
    
    if not bull_bear_df.empty:
        export_table_csv(bull_bear_df, EXPORTS_DIR / "regime_bull_bear.csv")
        export_table_latex(bull_bear_df, EXPORTS_DIR / "regime_bull_bear.tex", float_format="%.4f")
    
    if not vol_df.empty:
        export_table_csv(vol_df, EXPORTS_DIR / "regime_volatility.csv")
        export_table_latex(vol_df, EXPORTS_DIR / "regime_volatility.tex", float_format="%.4f")


## Summary Statistics


In [ ]:
if regime_df is not None:
    print("=== Regime Performance Summary ===\n")
    
    # Bull/Bear summary
    bull_bear_df = regime_df[regime_df['regime_type'] == 'bull_bear']
    if not bull_bear_df.empty:
        print("Bull/Bear Regimes:")
        print(f"  Best Sharpe: {bull_bear_df.loc[bull_bear_df['sharpe'].idxmax(), 'regime_label']} ({bull_bear_df['sharpe'].max():.4f})")
        print(f"  Best Return: {bull_bear_df.loc[bull_bear_df['total_return'].idxmax(), 'regime_label']} ({bull_bear_df['total_return'].max():.4f})")
        print(f"  Lowest Drawdown: {bull_bear_df.loc[bull_bear_df['max_drawdown'].idxmin(), 'regime_label']} ({bull_bear_df['max_drawdown'].min():.4f})")
        print()
    
    # Volatility summary
    vol_df = regime_df[regime_df['regime_type'] == 'volatility']
    if not vol_df.empty:
        print("Volatility Regimes:")
        print(f"  Best Sharpe: {vol_df.loc[vol_df['sharpe'].idxmax(), 'regime_label']} ({vol_df['sharpe'].max():.4f})")
        print(f"  Best Return: {vol_df.loc[vol_df['total_return'].idxmax(), 'regime_label']} ({vol_df['total_return'].max():.4f})")
        print(f"  Lowest Drawdown: {vol_df.loc[vol_df['max_drawdown'].idxmin(), 'regime_label']} ({vol_df['max_drawdown'].min():.4f})")
        print()
    
    # Overall best
    best_overall = regime_df.loc[regime_df['sharpe'].idxmax()]
    print(f"Overall Best Regime: {best_overall['regime_type']} - {best_overall['regime_label']}")
    print(f"  Sharpe: {best_overall['sharpe']:.4f}")
    print(f"  Total Return: {best_overall['total_return']:.4f}")
    print(f"  Max Drawdown: {best_overall['max_drawdown']:.4f}")
